In [ ]:
from dataclasses import dataclass
from enum import Enum
from itertools import product
from typing import List
from typing import Tuple
from typing import Union
import math

import matplotlib.pyplot as plt

In [ ]:
# Constants
ALPHABET = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

SQUARE_3 = [
    [4, 9, 2],
    [3, 5, 7],
    [8, 1, 6],
]
SQUARE_4 = [
    [4, 14, 15, 1],
    [9, 7, 6, 12],
    [5, 11, 10, 8],
    [16, 2, 3, 13],
]
SQUARE_7 = [
    [22, 47, 16, 41, 10, 35, 4],
    [5, 23, 48, 17, 42, 11, 29],
    [30, 6, 24, 49, 18, 36, 12],
    [13, 31, 7, 25, 43, 19, 37],
    [38, 14, 32, 1, 26, 44, 20],
    [21, 39, 8, 33, 2, 27, 45],
    [46, 15, 40, 9, 34, 3, 28],
]

SQUARES = {
    3: SQUARE_3,
    4: SQUARE_4,
    7: SQUARE_7,
}

In [ ]:
# Utilities

def str_to_number(ch: str) -> int:
    """Convert character to a number"""
    i = ALPHABET.find(ch.upper())
    if i < 0:
        return 0

    mag =  10 ** (i // 9)
    offset = (i % 9) + 1
    return mag * offset


REVERSE_ALPHABET = {str_to_number(ch): ch for ch in ALPHABET}


def number_to_str(n: int) -> str:
    """Convert a number back to text"""
    x1 = 100 * (n // 100)
    x2 = 10 * ((n % 100) // 10)
    x3 = n % 10

    output = REVERSE_ALPHABET.get(x1, '')
    output += REVERSE_ALPHABET.get(x2, '')
    output += REVERSE_ALPHABET.get(x3, '')

    return output


class Option(Enum):
    NEVER = 0
    MAYBE = 1
    ALWAYS = 2


@dataclass(init=False, unsafe_hash=True)
class Character:
    num: int
    text: str

    def __init__(self, num: int = 0, text: str = ''):
        """Construct from number and/or text"""
        self.num = num
        self.text = text
        if num == 0:
            self.num = str_to_number(text)
        elif len(text) == 0:
            self.text = number_to_str(num)

    # Operators

    def __floordiv__(self, other: float):
        """Floor divide by number"""
        return Character(num=self.num // other, text=self.text)

    def __add__(self, other):
        """Add Character"""
        a, b = self, other
        if a < b:
            b, a = a, b
        return Character(num=a.num + b.num, text=a.text + b.text)

    def __lt__(self, other):
        """Comparison operator"""
        if isinstance(other, Character):
            return self.num < other.num
        return self.num < other

    def __gt__(self, other):
        """Comparison operator"""
        if isinstance(other, Character):
            return self.num > other.num
        return self.num > other

    def __eq__(self, other):
        """Equality operator"""
        if isinstance(other, Character):
            return self.num == other.num
        return self.num == other

    def __str__(self):
        """String operator"""
        return self.text


class NumSeq:
    def __init__(self, numbers: Union[str, Tuple[Character]]):
        """Construct from tuple of numbers or string"""
        if isinstance(numbers, str):
            self.numbers = [Character(text=ch) for ch in numbers]
            self.numbers = tuple([n for n in self.numbers if n.num > 0])
        else:
            self.numbers = tuple(numbers)

    def compatible(self, size: int) -> bool:
        """Check if sequence is compatible with square of given size"""
        for ch in self.numbers:
            if ch > size * size:
                return False

        return True

    # Create children

    def remove_duplicates(self, option: Option = Option.MAYBE) -> List:
        """Possibly remove duplicates"""
        if option == Option.NEVER:
            return [self]

        dupe_i = []
        for i, ch in enumerate(self.numbers):
            if ch in self.numbers[:i]:
                dupe_i.append(i)

        choices = [[True] * len(dupe_i)]
        if option == Option.MAYBE:
            choices = product([False, True], repeat=len(dupe_i))

        output = []
        for choice in choices:
            numbers = []
            last_i = 0
            for i, drop in zip(dupe_i, choice):
                numbers += self.numbers[last_i:i]
                if not drop:
                    numbers.append(self.numbers[i])
                last_i = i + 1
            numbers += self.numbers[last_i:]
            output.append(NumSeq(numbers))

        return output

    def _reduce_one(self, ch: Character) -> List[Character]:
        """Reduce one character"""
        output = {ch // 100, ch // 10, ch}
        output.discard(0)
        return list(sorted(output))

    def reduce(self, size: int, option: Option = Option.MAYBE) -> List:
        """Possibly reduce numbers by an order of magnitude"""
        if option == Option.NEVER:
            return [self]

        choices = [self._reduce_one(ch) for ch in self.numbers]

        if option == Option.ALWAYS:
            choices = [[choice[0]] for choice in choices]

        output = [NumSeq(choice) for choice in product(*choices)]
        output = [o for o in output if o.compatible(size)]

        return output

    def _combine_at(self, size: int, from_i: int) -> List:
        """Possibly combine adjacent numbers at given index"""
        if from_i + 1 >= len(self.numbers):
            return [self]

        x1 = self.numbers[from_i]
        x2 = self.numbers[from_i + 1]

        mag1 = math.floor(math.log(x1.num, 10))
        mag2 = math.floor(math.log(x2.num, 10))

        if mag1 <= mag2 or x1 + x2 > size * size:
            return [self]

        new_numbers = self.numbers[:from_i] + (x1 + x2,) + self.numbers[from_i + 2:]
        return [self, NumSeq(new_numbers)]
    
    def combine(self, size: int, option: Option = Option.MAYBE) -> List:
        """Possibly combine adjacent numbers (20 + 1 -> 21)."""

        output = [self]
        if option == option.NEVER:
            return output

        for i in reversed(range(len(self.numbers))):
            output = [o2 for o in output for o2 in o._combine_at(size, i)]

        if option == option.ALWAYS:
            min_size = min([len(o.numbers) for o in output])
            output = [o for o in output if len(o.numbers) == min_size]
        
        return output

    def get_children(
        self,
        size: int,
        remove_duplicates: Option = Option.MAYBE,
        reduce: Option = Option.MAYBE,
        combine: Option = Option.MAYBE
    ) -> List:
        """Get all possible children of the sequence"""
        output = [self]
        output = [o2 for o in output for o2 in o.remove_duplicates(option=remove_duplicates)]
        output = [o2 for o in output for o2 in o.reduce(size, option=reduce)]
        output = [o2 for o in output for o2 in o.combine(size, option=combine)]

        return output

    # Rendering

    def draw(self, square: List[List[int]], ax, show=True):
        """Draw a single sequence on a square"""
        coord_map = dict()
        size = len(square)
        for i, row in enumerate(square):
            for j, ch in enumerate(row):
                coord_map[ch] = (j + 0.5, size + 0.5 - i)

        xs = []
        ys = []
        if self.compatible(size):
            xs = [coord_map[ch.num][0] for ch in self.numbers]
            ys = [coord_map[ch.num][1] for ch in self.numbers]

        label = ' '.join([str(ch.num) for ch in self.numbers])
        
        ax.plot(xs, ys, 'o-r')

        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_xlim([0, size + 1])
        ax.set_ylim([0, size + 1])
        ax.set_title(f'{str(self)}\n{label}', pad=0, fontsize=10)

        if show:
            plt.show()

    @staticmethod
    def draw_all(sequences: List, square: List[List[int]]):
        """Draw all sequences on a square"""
        cols = 5
        rows = ((len(sequences) - 1) // cols) + 1
        
        plt.figure(figsize=(15, 15 * rows / cols))

        for i, seq in enumerate(sequences):
            ax = plt.subplot(rows, cols, i + 1)
            seq.draw(square, ax, show=False)

        plt.show()

    # Operators

    def __repr__(self) -> str:
        """repr operator"""
        return f'NumSeq({tuple((n.num for n in self.numbers))})'

    def __str__(self) -> str:
        """string operator"""
        return ' '.join([n.text for n in self.numbers])

    def __hash__(self) -> int:
        """hash operator"""
        return hash(self.numbers)

    def __eq__(self, other) -> bool:
        """"Equality operator"""
        if not isinstance(other, NumSeq):
            return False
        return self.numbers == other.numbers

In [341]:
TEXT = 'Nero Caesar'
SIZE = 7

In [ ]:
seq = NumSeq(TEXT)
NumSeq.draw_all(seq.get_children(SIZE), SQUARES[SIZE])